# import library

In [189]:
import json
import requests
from bs4 import BeautifulSoup
from collections import defaultdict

# Get html

In [82]:
def get_html(url):#获得页面html代码
    req = requests.get(url)
    html = req.text
    return html

def crawl_list(url, pattern):
    html = get_html(url)
    soup = BeautifulSoup(html, 'lxml')
    title = soup.find('main')


In [ ]:

url = "https://neurips.cc/Conferences/2019/AcceptedPapersInitial"
html = get_html(url)

# Find target elements

In [183]:
# Suppose I can always use three step to find target elements
# 1. Find the first element contains paper
# 2. Find the container contains all papers, For ICLR which uses different containers for oral/poster/reject, just find all oral papers, or all poster papers in one step.
# 3. List those elements

def find_target_elements(html, pattern):
    eles = pattern.split("/")
    soup = BeautifulSoup(html, 'lxml')
    lists = soup.select(eles[0])
    for l in lists:
        if l.text == first:
            break

    if eles[1] == '..':
        container = l
        for i in range(len(eles[0].split(">"))):
            container = container.parent
    if '@' in eles[2]:
        selector, index = eles[2].split("@")
    container_lists = container.select(selector)
    if ':' not in index:
        nodes = container_lists[int(index) - 1]
    else:
        start, end = index.split(":")
        start = (int(start) - 1) if start else 0
        end = (int(end) - 1) if end else len(container_lists)
        nodes = container_lists[start:end]
    return nodes

In [184]:

nodes = find_target_elements(html, "p>b/../p@3:")

# Parser

Usually the regular expression is not power enough

In [169]:
def author_parser(text):
    institute_list = []
    left = 0
    institute = ""
    for c in text:
        if c == '(':
            left += 1
            if left == 1:
                institute = institute.strip()
                institute_list.append(institute)
                institute = ""
        if c == ')':
            left -= 1
            continue
        if left > 0 or c == '·':
            continue
        institute += c
    return institute_list


In [185]:

def institute_parser(text):
    institute_list = []
    left = 0
    institute = [""]
    for c in text:
        if c == '(':
            left += 1
            if left == 1:
                continue
        elif c == ')':
            left -= 1
            if left > 0:
                continue
            if left == 0:
                institute[-1] = institute[-1].strip()
                institute_list.append(institute)
                institute = [""]
        if left == 0 or left > 1:
            continue
        if c in ['&', '/']:
            institute[-1] = institute[-1].strip()
            institute.append('')
            continue
        institute[-1] += c
    return institute_list

# Extract item

In [186]:
def extract_item(element, rules):
    paper_item = {}
    extra_item = []
    for key in rules:
        if isinstance(rules[key], str):
            paper_item[key] = element.select(rules[key])[0].text
        elif isinstance(rules[key], list):
            text = element.select(rules[key][0])[0].text

            # Let's only consider author and/or institute
            if key == 'authors':
                if isinstance(rules[key][1]['author'], str):
                    author_list = re.findall(rules[key][1]['author'], text)
                else:
                    author_list = rules[key][1]['author'](text)
                if isinstance(rules[key][1]['institute'], str):
                    institute_list = re.findall(rules[key][1]['institute'], text)
                else:
                    institute_list = rules[key][1]['institute'](text)
                item = tuple(zip(author_list, institute_list))
                assert len(author_list) == len(institute_list)
            else:
                print("not authors")
                raise NotImplementedError
            paper_item[key] = item
    return paper_item

# (Facebook AI research (FAIR)) makes regular expression fail
rules = {'title': 'p>b', 'authors': ['i', {'author': ' ?([^(·]+) \([^(]+\)', 'institute': '[^(·]+ \(([^(]+)\)'}]}
rules = {'title': 'p>b', 'authors': ['i', {'author': author_parser, 'institute': institute_parser}]}

paper_set = {}
for element in nodes:
    item = extract_item(element, rules)
    title = item.pop('title')
    paper_set[title] = item

In [187]:
len(paper_set)

1429

In [191]:
def main(url):

    html = get_html(url)

    nodes = find_target_elements(html, "p>b/../p@3:")
    rules = {'title': 'p>b', 'authors': ['i', {'author': author_parser, 'institute': institute_parser}]}

    paper_set = {}
    for element in nodes:
        item = extract_item(element, rules)
        title = item.pop('title')
        paper_set[title] = item
    return paper_set

url = "https://neurips.cc/Conferences/2019/AcceptedPapersInitial"
paper_set = main(url)
print(len(paper_set))

with open('data/neurips2019.json', 'w') as fp:
    json.dump(paper_set, fp, indent=4)

1429
